In [2]:
import numpy as np
import os
import cobra
from pathlib import Path
from sklearn.metrics import mean_squared_error
import pandas as pd
from cobra.io import load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
import scipy.io as scio
from scipy import sparse
import cobra.util
from cobra import Model,Reaction,Metabolite

path = "F:\python\Bayesian_python"
os.chdir(path)

from code0824 import abc_python_max
from code_generation_preparation import getrSample
from code_generation_preparation import sumBiomass
from code_generation_preparation import updateprior



species = "Saccharomyces_cerevisiae"
generation = 150
enzymedataFile = "data/Saccharomyces_cerevisiae_dl.mat"
#enzymedataFile = "data/model.mat"
z = scio.loadmat(enzymedataFile)
enzymedata = z['enzymedata'][0,0]
max_growth = z['max_growth']
growthdata = z['growthdata']
model = z['model'][0,0]
strain = z['strain']
rxn2block = z['rxn2block']

#load the model in cobra format
scio.savemat('data/model.mat', {'model': model})
model_cobra = load_matlab_model('data/model.mat')


import scipy.io as io
F = sumBiomass(model,model_cobra)
#calculate the prot_weight
#F = 0.46  #怎么从sumBiomass中导出结果，matlab版本为0.46
tot_prot_weight = F[0]*0.5
if strain == 'Kluyveromyces_marxianus':
    tot_prot_weight = 0.325
elif strain == 'Kluyveromyces_lactis':
    tot_prot_weight = 0.245

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-26


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, ce, e, er, erm, g, gm, lp, m, mm, n, p, v, vm


P -> [[0.46000001]] g/gDW
X ->[[0.46000001]] gDW/DW
Growth = 0.08158852335573963 1/h


In [25]:
    met = cobra.Metabolite ('cost' , name='cost' , compartment='c')
    model_cobra.add_metabolites (met)
    model_cobra.add_boundary (model_cobra.metabolites.get_by_id ("cost") , type="sink" , ub=tot_prot_weight * 1000 , lb=0)
    #nMets = len (model_cobra.metabolites)
    nRxns = len (model_cobra.reactions)
    cost_list = np.zeros ((nRxns , 1))
    kcat_random = enzymedata['kcat']
    prot_cost_info_value = [enzymedata["MW"][i] / kcat_random[i] for i in range (len (kcat_random))]
    prot_cost_info_value = [item[0] for item in prot_cost_info_value]
    prot_cost_info_id = [str (item[0][0]) for item in enzymedata["rxn_list"]]
    prot_cost_info = dict (zip (prot_cost_info_id , prot_cost_info_value))
    for p , rxnid in enumerate (model['rxns']):
        rxnid = tuple (rxnid)
        cost = prot_cost_info.get (rxnid[0][0] , 0)
        cost_list[p , 0] = cost
    cost_floats = [float (m) for m in cost_list]
    for q , reaction in enumerate (model_cobra.reactions):
        reaction.add_metabolites ({'cost': cost_floats[q]})

    proc = 18
    numPerGeneration = 126 #126/18 = 7
    rejectnum = 0.2
    generation = 100

    if len(max_growth) == 0 and len(growthdata) == 0:
        max_growth = [strain,'D-glucose',0.2,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,'aerobic','Batch','MIN']
        simulated_3 = abc_python_max(model,enzymedata,enzymedata['kcat'],tot_prot_weight,growthdata,max_growth,1,1,1,rxn2block)
        simulated_3_tmp = np.array(np.array(simulated_3)[2][0])[0,1]
        if simulated_3_tmp >0.2:
            max_growth = [strain,'D-glucose',simulated_3_tmp,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,'aerobic','Batch','MIN']


In [9]:

    output = 0

    D = abc_python_max (model , model_cobra , enzymedata , enzymedata["kcat"] , tot_prot_weight , growthdata ,
                        max_growth , 1 , 1 , 1 , rxn2block,output)[0]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [26]:
        objective = 'r_2111'
        osenseStr = 'max'
        from code0824 import rmsecal
        if len (growthdata)!=0:
            rmse_1 , exp_1 , simulated_1 = rmsecal (model , model_cobra , growthdata , True , objective , osenseStr ,
                                                    prot_cost_info_id , prot_cost_info_value ,prot_cost_info, tot_prot_weight ,
                                                    rxn2block)
        else:
            rmse_1 = []
            exp_1 = []
            simulated_1 = []
        # print("RMSE_1 finish !")
        # second searcch for maxmial growth rate without constrain
        if len (max_growth)!=0:
            rmse_2 , exp_2 , simulated_2 = rmsecal (model , model_cobra , max_growth , False , objective , osenseStr ,
                                                    prot_cost_info_id , prot_cost_info_value ,prot_cost_info, tot_prot_weight ,
                                                    rxn2block)
        else:
            rmse_2 = []
            exp_2 = []
            simulated_2 = []
        #end_time = time.time ()
        #execution_time = end_time - start_time
        #print ("execution time: " , execution_time , " seconds")

No. 1finish !
No. 2finish !
No. 3finish !
No. 4finish !
No. 5finish !
No. 6finish !
No. 7finish !
No. 8finish !
No. 9finish !
No. 10finish !
No. 11finish !
No. 12finish !
No. 13finish !
No. 14finish !
No. 15finish !
No. 16finish !
No. 17finish !
RMSE = 0.9819214215183005
No. 1finish !
No. 2finish !
No. 3finish !
No. 4finish !
No. 5finish !
No. 6finish !
No. 7finish !
No. 8finish !
RMSE = 1.8940209096371767


In [27]:
        exp = np.array ([exp_1 , exp_2])
        simulated = np.array ([simulated_1 , simulated_2])
        rmse = np.array ([rmse_1 , rmse_2])
        rmse_final = np.nanmean (rmse , 0)
        rmse_final

C:\Users\sherr\AppData\Local\Temp\ipykernel_11288\4076241837.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  exp = np.array ([exp_1 , exp_2])
C:\Users\sherr\AppData\Local\Temp\ipykernel_11288\4076241837.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  simulated = np.array ([simulated_1 , simulated_2])


1.4379711655777387

In [24]:
tmp = pd.read_csv('result/kcat_genra100.txt',delimiter=',',header=None)
kcat_100 = np.array(tmp)
kcat_100 = kcat_100[0:-2, :]
[a, b] = updateprior(kcat_100)
enzymedata['kcat'] = np.transpose(a)
enzymedata['kcat_var'] = np.transpose(b)

In [ ]:
        if nfound > 0:
            tmp = pd.read_csv('result/'+'kcat_genra' + str(output) + '.txt', delimiter=',',header= None)
            tmp = np.array(tmp)
            theta_100 = tmp[-1, :]
            kcat_100 = tmp[0:-2, :]
            tot_prot_weight = tmp[-2, 0]
            sampledgeneration = output+1
            # recalculate the sigma and mu
            [a, b] = updateprior(kcat_100)
            enzymedata['kcat'] = np.transpose(a)
            enzymedata['kcat_var'] = np.transpose(b)
        if sampledgeneration <= generation:
            print("No. " + str(output) + ' generation')

            # generate a
            old = theta_100
            kcat_old_100 = kcat_100

            # repeat a generation
            if sampledgeneration == 1:
                sample_generation = 144
            else:
                sample_generation = numPerGeneration

            # generate one generation sample of kcats
            kcat_random_all = getrSample(enzymedata['kcat'], enzymedata['kcat_var'], enzymedata['enzyme_ec_kcat_range'][:, 0],enzymedata['enzyme_ec_kcat_range'][:, 1], sample_generation,method = 'normal')
            print("kcat random finish !")
            #print(kcat_random_all)
            # start sampling
            new_tmp = np.zeros((18, 7))
            # Use a Pool to parallelize the work among 20 cores
            num_cpus = 17
            print(num_cpus)
            with Pool (processes=num_cpus) as pool:
                args = [(
                        i , output , model , model_cobra , enzymedata , kcat_random_all , tot_prot_weight , growthdata ,
                        max_growth , proc , sample_generation , rxn2block) for i in range (int (proc))]
                results = pool.starmap (parallel_task , args)
            new_tmp = np.vstack (results)
            print(new_tmp)
            print ("RMSE calculation finish !")

            # find the best 100 samples
            new = new_tmp
            theta = np.append (old , new)
            kcat = np.append (kcat_old_100 , kcat_random_all , axis=1)

            # initialize an empty set to store the best 100 after each step
            theta_100 = np.sort (theta.flatten ())[:100]
            D = abs (theta_100[99] - theta_100[0])
            D_idx = np.argsort (theta.flatten ())[:100]
            D_100 = theta_100[99]
            kcat_100 = kcat[: , D_idx]
            tot = np.tile(tot_prot_weight, (1, len(theta_100)))
            kcat_genra = []
            for m in range(len(kcat_100[0,:])):
                for n in range(len(tot[0,:])):
                    if m == n:
                        for k in range(len(theta_100)):
                            if n == k:
                                t = np.append(kcat_100[:,m], tot[:,n])
                                t = np.append(t, theta_100[k])
                                kcat_genra.append(t)
            kcat_genra = np.transpose (kcat_genra)
            path = 'result'
            os.makedirs (path , exist_ok=True)
            filename = 'kcat_genra' + str (output+1) + '.txt'
            full_path = os.path.join (path , filename)
            np.savetxt (full_path , kcat_genra,delimiter=",", fmt="%f")

            # recalclate the sigma and mu
            [a , b] = updateprior (kcat_100)
            enzymedata['kcat'] = np.transpose(a)
            enzymedata['kcat_var'] = np.transpose(b)

In [55]:
model1 = read_sbml_model("ecYeastGEM.xml")
model2 = read_sbml_model("ecYeast_DL_update_some_kcat.xml")
model3 = read_sbml_model("scer_test_model.xml")
model4 = model_cobra.copy()

Read LP format model from file C:\Users\sherr\AppData\Local\Temp\tmpj6t2h641.lp
Reading time = 0.04 seconds
: 2743 rows, 12418 columns, 48160 nonzeros


In [ ]:
metabolites1 = set([m.id for m in ecModel.metabolites])
metabolites2 = set([m.id for m in model_cobra.metabolites])
reactions1 = set([r.id for r in ecModel.reactions])
reactions2 = set([r.id for r in model_cobra.reactions])

# 找出只在一个模型中存在的代谢物和反应
unique_metabolites1 = metabolites1 - metabolites2
unique_metabolites2 = metabolites2 - metabolites1
unique_reactions1 = reactions1 - reactions2
unique_reactions2 = reactions2 - reactions1

# 输出结果
print("Unique metabolites in model1:", unique_metabolites1)
print("Unique metabolites in model2:", unique_metabolites2)
print("Unique reactions in model1:", unique_reactions1)
print("Unique reactions in model2:", unique_reactions2)

In [152]:
#原版代码

def constrainAbandance(model,measured):
    '''
    model       : eModel from convertToEnzymeModel()
    measured    : a dictionary with measured enzyme abandance, in the unit of mmol/gdw
    g
    # define the upper bound of protein exchange reactions with protein abandance.
    # e.g. the reaction id is in the format of "prot_TD01GL001367_exchange"

    Usage: model = constrainAbandance(model,MWs, non_measured,UB)
    '''

    for prot_id, ab in measured.items():
        rxn_id = 'prot_{0}_exchange'.format(prot_id)
        model.reactions.get_by_id(rxn_id).upper_bound = ab

    return model

def constrainPool(model,MWs, measured, non_measured,UB,copy=True):
    '''

    model       : eModel from convertToEnzymeModel()
    MWs         : a dictionary with molecular weight of enzymes, in the unit of kDa
    non_measured: a list of enzymes without proteomics data
    measured    : a dictionary with measured enzyme abandance, in the unit of mmol/gdw
    UB          : upper bound for the combined pool of those non_measured enzymes
    copy        : if creat a copy of the original model

    Define new rxns: For each enzyme, add a new rxn that draws enzyme from the
    enzyme pool (a new metabolite), and remove previous exchange rxn. The new
    rxns have the following stoichiometry (T is the enzyme pool):
     MW[i]*P[T] -> P[i]

    Usage: model = constrainPool(model,MWs, non_measured,UB)

    Gang Li, last updated 2020-03-04
    '''
    if copy: model = model.copy()
    # create prot_pool metabolite
    prot_pool = Metabolite('prot_pool')
    prot_pool.name = prot_pool.id

    rxns_to_add  = list()
    rxns_to_drop = list()
    for prot in non_measured:
        #prot_exchange_rxn = model.reactions.get_by_id('prot_{0}_exchange'.format(prot))
        prot_exchange_rxn = model.reactions.get_by_id('prot_{0}'.format(prot[0]))

        draw_rxn = Reaction('draw_prot_{0}'.format(prot))
        draw_rxn.name = draw_rxn.id
        draw_rxn.add_metabolites({prot_pool:-MWs[prot],list(prot_exchange_rxn.metabolites)[0]:1})

        rxns_to_add.append(draw_rxn)
        rxns_to_drop.append(prot_exchange_rxn)

    # add draw reaction into model
    model.add_reactions(rxns_to_add)
    model.remove_reactions(rxns_to_drop)

    # change the upper bound for all reactions as np.inf
    for rxn in model.reactions: rxn.upper_bound = np.inf

    # add prot_pool_exchange rxn
    rxn_prot_pool_exg = Reaction('prot_pool_exchange')
    rxn_prot_pool_exg.name = rxn_prot_pool_exg.id
    rxn_prot_pool_exg.add_metabolites({prot_pool:1})
    rxn_prot_pool_exg.lower_bound = 0
    rxn_prot_pool_exg.upper_bound = UB

    model.add_reaction(rxn_prot_pool_exg)

    # constrain the proteins with measure abandance
    constrainAbandance(model,measured)

    return model

def addEnzymesToRxn(rxn, kcat, protIDs, rxn_index=None):
    '''
    Add each enzyme as one of metabolites in the model, Current version does not support stoichiometric cofficients of subunits
    rxn      : the input Reaction object in cobrapy
    rxn_index: a integer like 1, for isoenzymes
    kcats    : kcat value for the reaction
    protIDs  : a single protein name, like "A", or a complex like "A and B". String
    MWs      : a dictionary with prot_id as key and molecular weight as value

    Usage: e_rxn, prot_exchange_rxns = addEnzymesToRxn(rxn, kcat, protIDs,MWs)

    Gang Li, last updated 2020-03-03
    '''

    e_rxn      = rxn.copy()
    if rxn_index is not None:
        e_rxn.id   = e_rxn.id + 'No{0}'.format(rxn_index)
        e_rxn.name = e_rxn.name + ' (No{0})'.format(rxn_index)
    prots = [item.strip() for item in protIDs.split('and')]


    # get compartment
    comp = None
    for met in rxn.metabolites:
        comp = met.compartment
        if rxn.get_coefficient(met)<0: comp = met.compartment

    # create Metabolite object for each protein and create exchange reaction
    prot_mets = []
    prot_exchange_rxns = []
    for prot in prots:
        prot_met = Metabolite('prot_{0}[{1}]'.format(prot,comp))
        prot_met.compartment =  comp
        prot_mets.append(prot_met)

        # add excange reaction of protein
        excg_rxn = Reaction('prot_{0}_exchange'.format(prot))
        excg_rxn.lower_bound = 0
        excg_rxn.gene_reaction_rule = prot
        excg_rxn.add_metabolites({prot_met:1})
        prot_exchange_rxns.append(excg_rxn)

    # add enzymes into reaction
    e_rxn.add_metabolites({prot_met:-1./kcat for prot_met in prot_mets})
    e_rxn.gene_reaction_rule = protIDs

    return e_rxn, prot_exchange_rxns

def convertToEnzymeModel(model,kcats):
    '''
    model .   : irrevModel
    kcats     : a dictionary with kcat values {('protein_id',rxn_id):100,...}

    Usage: eModel = convertToEnzymeModel(model,kcats)

    Gang Li, last updated 2020-03-04
    '''
    converted_reaction_list = []
    protein_exchange_rxns = {}
    for rxn in model.reactions:
        complexes = parse_gr_rule(rxn.gene_reaction_rule)

        # 1. for those reactions without genes
        if len(complexes) <1:
            converted_reaction_list.append(rxn)
            continue

        # 2. for those reactions with genes, but no kcat
        first_gene = [gene.id for gene in rxn.genes][0]
        if kcats.get((first_gene,rxn.id),None) is None:
            converted_reaction_list.append(rxn)
            continue

        # 3. for those reactions with isoenzymes, add arm reaction
        if len(complexes) >1:
            rxn_new, arm_rxn = getArmReaction(rxn)
            converted_reaction_list.append(arm_rxn)

            for i,complx in enumerate(complexes):
                prots = [item.strip() for item in complx.split('and')]
                kcat = kcats[(prots[0],rxn.id)]
                e_rxn, prot_exchange_rxns = addEnzymesToRxn(rxn_new, kcat, complx,rxn_index=i+1)

                converted_reaction_list.append(e_rxn)
                for prot_exchange_rxn in prot_exchange_rxns: protein_exchange_rxns[prot_exchange_rxn.id] = prot_exchange_rxn

            continue

        if len(complexes) == 1:
            complx = complexes[0]
            prots = [item.strip() for item in complx.split('and')]
            kcat = kcats[(prots[0],rxn.id)]
            e_rxn, prot_exchange_rxns = addEnzymesToRxn(rxn, kcat, complx,rxn_index=1)
            converted_reaction_list.append(e_rxn)
            for prot_exchange_rxn in prot_exchange_rxns: protein_exchange_rxns[prot_exchange_rxn.id] = prot_exchange_rxn

    eModel = Model()
    eModel.add_reactions(converted_reaction_list)
    eModel.add_reactions(protein_exchange_rxns.values())
    eModel.enzymes = set([exgrxn.split('_')[1] for exgrxn in protein_exchange_rxns.keys()])
    print('Number of enzymes:', len(eModel.enzymes))

    return eModel

def getArmReaction(rxn):
    '''
    Adapted from addArmReaction.m from geckomat. Add an arm reaction for the selected reaction in the model.

    rxn: the reaction Object in cobrapy

    Original reaction: A + B --> C + D

    Arm reaction    : A + B --> pmet   (no gr rule)
    Change the orginial reaction to:  pmet --> C + D (use old gr rule)

    The arm reaction has a id format of "arm_rxnID" and a name format of "rxnName (arm)"

    The intermediate metabilite has a name format of "pmet_rxnID"

    The arm reaction shares the same lb, ub, gr rules, subsystems with original reaction.

    Compartment: fistly try to use the same compartment as substrates, then products', otherwise None.


    Usage: rxn_new, arm_rxn = addArmReaction(model,rxn_id).

    Gang Li, Last update: 2020-03-03
    '''

    # 1. create intermediate metabilite
    rxnID = rxn.id
    comp = None
    for met in rxn.metabolites:
        comp = met.compartment
        if rxn.get_coefficient(met)<0: comp = met.compartment

    pmet = Metabolite('pmet_{0}'.format(rxnID),compartment=comp)

    # 2. create arm reaction:
    arm_rxn                    = Reaction('arm_{0}'.format(rxnID))
    arm_rxn.name               = rxn.name + ' (arm)'
    arm_rxn.subsystem          = rxn.subsystem
    arm_rxn.lower_bound        = rxn.lower_bound
    arm_rxn.upper_bound        = rxn.upper_bound
    arm_rxn.gene_reaction_rule = ''

    mets = {met:rxn.get_coefficient(met) for met in rxn.metabolites if rxn.get_coefficient(met)<0}
    mets[pmet] = 1

    arm_rxn.add_metabolites(mets)

    # 3. change orignal reaction to pmet --> C + D
    rxn_new = rxn.copy()
    rxn_new.subtract_metabolites({met:rxn_new.get_coefficient(met) for met in rxn_new.metabolites if rxn_new.get_coefficient(met)<0})
    rxn_new.add_metabolites({pmet:-1})

    return rxn_new, arm_rxn

def parse_gr_rule(gr):
    '''
    Parse gr rule into a list of components.
    gr: gene reaction rule, defined in cobrapy.

    For example:

    Input         : Output
    A or B        : ["A", "B"]
    (A and B)     : ["A and B"]
    (A and B) or C: ["A and B","C"]

    Usage: complexes = parse_gr_rule(gr)

    Gang Li, last updated 2020-03-04

    '''
    complexes = [item.strip().replace('(','').replace(')','') for item in gr.split('or')]
    if len(complexes) < 2 and len(complexes[0]) < 1: complexes = []

    return complexes

In [140]:
ecpost = "emodel_Saccharomyces_cerevisiae_Posterior_mean.mat"
#enzymedataFile = "data/model.mat"
ecpost = scio.loadmat(ecpost)
ecpost['proteins']

KeyError: 'proteins'

In [157]:
import os
import pandas as pd

# Create a list to store the last lines of each file
last_lines = []
column_names = []

# Iterate through all the kcat_genra*.txt files
for i in range(1, 42):  # Assuming the files are numbered from 1 to 40
    file_name = f"result/kcat_genra{i}.txt"
    column_names.append(str(i))

    # Read the last line of each file
    with open(file_name, 'r') as file:
        last_line = file.readlines()[-1]

    # Convert the last line to a list of values
    last_line_values = last_line.strip().split(',')

    last_lines.append(last_line_values)

# Convert the list of last lines into a DataFrame
df = pd.DataFrame(last_lines, columns=column_names)

# Save the DataFrame to an Excel file
excel_path = 'last_lines_kcat_genra.xlsx'
df.to_excel(excel_path, index=False)


ValueError: 41 columns passed, passed data had 100 columns

In [159]:
import pandas as pd

# Create an empty DataFrame to store the last lines
df = pd.DataFrame()

# Iterate through all the kcat_genra*.txt files
for i in range(1, 41):  # Assuming the files are numbered from 1 to 40
    file_name = f"result/kcat_genra{i}.txt"

    # Read the last line of each file
    with open(file_name, 'r') as file:
        last_line = file.readlines()[-1]

    # Convert the last line to a list of floating-point numbers
    last_line_values = [float(x) for x in last_line.strip().split(',')]

    # Add the last line values to the DataFrame as a new column
    df[f"{i}"] = last_line_values

# Save the DataFrame to an Excel file
excel_path = 'last_lines_kcat_genra.xlsx'
df.to_excel(excel_path, index=False)


In [66]:
species = "Saccharomyces_cerevisiae"
generation = 150
enzymedataFile = "data/emodel_Saccharomyces_cerevisiae_dl.mat"
#enzymedataFile = "data/model.mat"
zz = scio.loadmat(enzymedataFile)

emodel = zz['emodel'][0,0]


#load the model in cobra format
#scio.savemat('data/emodel.mat', {'emodel': emodel})
#model_cobra2 = load_matlab_model('data/emodel.mat')
model_cobra2 = read_sbml_model('data/emodel_Saccharomyces_cerevisiae_DL.xml')
output = 0
D = abc_python_max (model , model_cobra2 , enzymedata , enzymedata["kcat"] , tot_prot_weight , growthdata ,
                        max_growth , 1 , 1 , 1 , rxn2block,1)
D

nstep:1/1


C:\Users\sherr\AppData\Local\Temp\ipykernel_2776\423083270.py:27: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  exp = exp.astype (np.float)


No. 1finish !


C:\Users\sherr\AppData\Local\Temp\ipykernel_2776\423083270.py:27: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  exp = exp.astype (np.float)


No. 2finish !


...

No. 7finish !


C:\Users\sherr\AppData\Local\Temp\ipykernel_2776\423083270.py:27: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  exp = exp.astype (np.float)


No. 8finish !
RMSE = 4.166851739822311
execution time:  52.10885310173035  seconds
rmse_final is  [[6.11104134]]


C:\Users\sherr\AppData\Local\Temp\ipykernel_2776\423083270.py:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  exp = np.array ([exp_1 , exp_2])
C:\Users\sherr\AppData\Local\Temp\ipykernel_2776\423083270.py:181: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  simulated = np.array ([simulated_1 , simulated_2])


(array([[6.11104134]]),
 array([array([[ 2.50000000e-02, -3.00000000e-01,  0.00000000e+00,
                 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                 0.00000000e+00,  7.51980000e-01, -7.31020000e-01],
               [ 5.00000000e-02, -6.00000000e-01,  0.00000000e+00,
                 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                 0.00000000e+00,  1.36970000e+00, -1.32510000e+00],
               [ 1.00000000e-01, -1.10000000e+00,  0.00000000e+00,
                 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                 0.00000000e+00,  2.40070000e+00, -2.31670000e+00],
               [ 1.50000000e-01, -1.70000000e+00,  0.00000000e+00,
                 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                 0.00000000e+00,  3.63790000e+00, -3.50660000e+00],
               [ 2.00000000e-01, -2.30000000e+00,  0.00000000e+00,
                 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                 0.00000000e+00,  

In [65]:
import numpy as np
import os
import cobra
from pathlib import Path
from sklearn.metrics import mean_squared_error
import pandas as pd
from cobra.io import load_matlab_model , save_matlab_model , read_sbml_model , write_sbml_model
import scipy.io as scio
from scipy import sparse
import cobra.util
from cobra import Model , Reaction , Metabolite
import cProfile
import time
from multiprocessing import cpu_count , Manager , Process
from code_generation_preparation import changeMedia
from code_generation_preparation import anaerobicModel

def rmsecal(model , model_cobra , data , constrain , objective , osenseStr , prot_cost_info_id , prot_cost_info_value ,prot_cost_info,
            tot_prot_weight , rxn2block):
    data = np.array (data)
    rmse_tmp = []
    simulated = np.zeros ((len (data[: , 0]) , 9))
    rxnNames = model['rxnNames']
    for i in range (len (data[: , 0])):
        exp = np.array (data[: , 2:11])  # u sub ace eth gly pyr ethyl_acetate co2 o2
        exp = exp * [1 , -1 , 1 , 1 , 1 , 1 , 1 , 1 , -1]
        exp = exp.astype (np.float)
        ex_mets = ['growth' , data[i , 1][0] + " exchange" , 'acetate exchange' , 'ethanol exchange' ,
                   'glycerol exchange' , 'pyruvate exchange' , 'ethyl acetate exchange' , 'carbon dioxide exchange' ,
                   'oxygen exchange']
        idx = []
        temp = []
        for k in range (len (ex_mets)):
            temp = np.where (rxnNames==ex_mets[k])
            idx.append (temp)
        idx = np.array (idx)
        idx = np.transpose (idx[: , 0])
        # Create a temp model
        start_time1 = time.time ()

        with model_cobra as model_tmp:

            # Suitable for different carbon sources
            #model_tmp = changeMedia (model , model_tmp , data[i , 1][0] , data[i , 15])  # 'D-glucose'
            model_tmp = changeMedia (model , model_tmp , 'D-glucose' , data[i , 15])
            model_tmp.reactions.get_by_id ('r_1634').bounds = 0 , 0
            model_tmp.reactions.get_by_id ('r_1631').bounds = 0 , 0

            # Suitable for anaerobic
            if data[i , 13]=="anaerobic" or data[i , 13]=="limited":
                model_tmp = anaerobicModel (model , model_tmp)

            if data[i , 13]=="limited":
                model_tmp.reactions.get_by_any ('oxygen exchange').lower_bound = -5

            model_tmp.reactions.get_by_id ('r_1714').lower_bound = 0

            id_substrate = np.where (model['rxnNames']==(data[i , 1][0] + " exchange"))
            if constrain==False:
                model_tmp.reactions[idx[0][1]].lower_bound = -1000
            else:
                model_tmp.reactions[idx[0][1]].lower_bound = exp[i , 1]


            # Solve the temp model
            sol_tmp = model_tmp.optimize ()
            sol = sol_tmp.fluxes  # sol[:,i] = sol_tmp.fluxes

        print ("No. " + str (i + 1) + "finish !")

        tmp = np.where (~np.isnan (exp[i]))[0]

        # Normalize the number of carbon
        excarbon = model['excarbon'][: , idx[0]]
        for x in range (len (idx[0 , :])):
            if excarbon[: , x]==0:
                excarbon[: , x] = 1
        exp_tmp = []
        for s in range (len (tmp)):
            exp_tmp.append (exp[i , tmp[s]] * excarbon[: , tmp[s]])
        sol_idx = np.transpose (sol[idx[0]])
        sol_idx = np.array (sol_idx)
        simulated_tmp = []
        for t in range (len (tmp)):
            simulated_tmp.append (
                np.array (sol_idx)[tmp[t]] * excarbon[: , tmp[t]])  # normalize the growth rate issue by factor 10

        # The expected blocked reactions
        exp_block = np.zeros ((1 , 218))  # 218怎么替换
        rxnblockidx_pre = np.setdiff1d (rxn2block , model['rxns'][idx[0][1]])
        rxnblockidx = []
        for k in range (len (model['rxns'])):
            if model['rxns'][k] in rxnblockidx_pre:
                rxnblockidx.append (k)

        simulated_block = []
        for t in range (len (rxnblockidx)):
            simulated_block.append (np.array (sol)[rxnblockidx[t]] * model['excarbon'][: , rxnblockidx[t]])
        id_zero = np.where (np.array (simulated_block)!=0)
        exp_block = exp_block[: , id_zero[0]]
        simulated_block = np.array (simulated_block)[id_zero[0]]
        exp_tmp = np.array (exp_tmp)
        # print ("exp_tmp: " , exp_tmp)
        # print ("simulated_tmp: " , simulated_tmp)
        # print ("exp_block: " , exp_block)
        # print ("simulated_block: " , simulated_block)

        # Calculate the RMSE
        if constrain:
            rmse_tmp.append (np.sqrt (mean_squared_error (np.append (exp_tmp , np.transpose (exp_block)) ,
                                                          np.append (simulated_tmp , np.transpose (simulated_block)))))
        else:
            if len (exp_tmp) >= 2:
                rmse_tmp.append (np.sqrt (mean_squared_error (exp_tmp[0:2] , np.array (simulated_tmp)[0:2])))
            else:
                rmse_tmp.append (np.sqrt (mean_squared_error (exp_tmp[0] , [simulated_tmp[0]])))
        simulated[i , :] = np.transpose (np.array (sol)[idx[0]])
        #print (rmse_tmp)
        #print (simulated[i , :])
    rmse = sum (rmse_tmp) / len (data[: , 0])
    print ("RMSE = " + str (rmse))

    # end_time = time.time ()
    # execution_time = end_time - start_time
    # print ("execution time: " , execution_time , " seconds")

    # return rmse, exp, simulated
    return rmse , exp , simulated

def abc_python_max(model , model_cobra , enzymedata , kcat_random_all , tot_prot_weight , growthdata , max_growth ,
                   proc , sample_generation , j , rxn2block , num):
    nstep = sample_generation / proc
    nstep = int (nstep)
    rmse_final = np.zeros ((1 , nstep))
    kcat_sample = kcat_random_all[: , ((j - 1) * nstep):(j * nstep)]

    # get carbonnum for each exchange rxn to further calculation of error
    if len (model["excarbon"])==0:  ##检查model中是否有excarbon, ~is_field
        model = addCarbonNum (model)  ##补写function
    for k in range (nstep):
        print ('nstep:' + str (k + 1) + '/' + str (nstep))
        kcat_random = kcat_sample[: , k]
        #prot_cost_info_value = [enzymedata["MW"][i] / kcat_random[i] for i in range (len (kcat_random))]
        #prot_cost_info_id = enzymedata["rxn_list"]
        prot_cost_info_value = [enzymedata["MW"][i] / kcat_random[i] for i in range (len (kcat_random))]
        prot_cost_info_value = [item[0] for item in prot_cost_info_value]
        prot_cost_info_id = [str(item[0][0]) for item in enzymedata["rxn_list"]]
        prot_cost_info = dict (zip (prot_cost_info_id , prot_cost_info_value))


        # first search with substrate constrain
        objective = 'r_2111'
        osenseStr = 'max'
        start_time = time.time ()

        if len (growthdata)!=0:
            rmse_1 , exp_1 , simulated_1 = rmsecal (model , model_cobra , growthdata , True , objective , osenseStr ,
                                                    prot_cost_info_id , prot_cost_info_value ,prot_cost_info, tot_prot_weight ,
                                                    rxn2block)
        else:
            rmse_1 = []
            exp_1 = []
            simulated_1 = []
        # print("RMSE_1 finish !")
        # second searcch for maxmial growth rate without constrain
        if len (max_growth)!=0:
            rmse_2 , exp_2 , simulated_2 = rmsecal (model , model_cobra , max_growth , False , objective , osenseStr ,
                                                    prot_cost_info_id , prot_cost_info_value ,prot_cost_info, tot_prot_weight ,
                                                    rxn2block)
        else:
            rmse_2 = []
            exp_2 = []
            simulated_2 = []
        end_time = time.time ()
        execution_time = end_time - start_time
        print ("execution time: " , execution_time , " seconds")

        # print("RMSE_2 finish !")

        exp = np.array ([exp_1 , exp_2])
        simulated = np.array ([simulated_1 , simulated_2])
        rmse = np.array ([rmse_1 , rmse_2])
        rmse_final[0 , k] = np.nanmean (rmse , 0)

        # only output simulated result for one generation
        if nstep!=1 or sample_generation!=1:
            simulated = []
            exp = []
        print ("rmse_final is " , rmse_final)

    return rmse_final , exp , simulated

